In [28]:
import xgboost
from numpy import loadtxt
import pandas as pd
from xgboost import XGBClassifier 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import pandas.util.testing as tm
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [4]:
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/non_ca_df_ev.csv',delimiter=",")
#df = pd.read_csv('/content/drive/MyDrive/MIDS/capstone/data/processed/non_ca_df_ev.csv') #swetha's path 

df = pd.read_csv('/content/df_health.csv')
df.head()
#df["year.y"].describe()
#list(df.dtypes)


,Unnamed: 0,GEOID,Region,Year,Renters,Eviction_Filings,Evictions,Eviction_Filings_RR,Evictions_RR,Tract,Total,Rent,Total_WHITE,Rent_WHITE,Total_BLACK,Rent_BLACK,Total_AI,Rent_AI,Total_ASIAN,Rent_ASIAN,Total_NHPI,Rent_NHPI,Total_OTHER,Rent_OTHER,Total_TWO,Rent_TWO,Total_WHITE_NL,Rent_WHITE_NL,Total_LATINX,Rent_LATINX,MedRent,MHHInc,rb_tot,rb_10.0,rb_14.9,rb_19.9,rb_24.9,rb_29.9,rb_34.9,rb_39.9,...,CHOLSCREEN_CrudePrev,CHOLSCREEN_Crude95CI,COLON_SCREEN_CrudePrev,COLON_SCREEN_Crude95CI,COPD_CrudePrev,COPD_Crude95CI,COREM_CrudePrev,COREM_Crude95CI,COREW_CrudePrev,COREW_Crude95CI,CSMOKING_CrudePrev,CSMOKING_Crude95CI,DENTAL_CrudePrev,DENTAL_Crude95CI,DIABETES_CrudePrev,DIABETES_Crude95CI,HIGHCHOL_CrudePrev,HIGHCHOL_Crude95CI,KIDNEY_CrudePrev,KIDNEY_Crude95CI,LPA_CrudePrev,LPA_Crude95CI,MAMMOUSE_CrudePrev,MAMMOUSE_Crude95CI,MHLTH_CrudePrev,MHLTH_Crude95CI,OBESITY_CrudePrev,OBESITY_Crude95CI,PAPTEST_CrudePrev,PAPTEST_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,population_count
0,0,8001007801,Denver,2016,1158,67.0,39.0,0.88,1.09,"Census Tract 78.01, Adams County, Colorado",1276,1111,598,504,257,235,7,7,211,198,0,0,175,139,28,28,248,172,521,467,742.0,25175.0,1111,20,45,88,91,91,159,23,...,56.7,"(54.0, 59.3)",35.2,"(32.5, 38.0)",7.4,"( 6.8, 8.0)",22.6,"(19.8, 25.8)",18.1,"(15.7, 20.9)",33.1,"(30.1, 35.8)",32.9,"(29.5, 37.0)",11.5,"(10.9, 12.0)",33.4,"(32.2, 34.7)",2.7,"( 2.5, 2.9)",35.1,"(32.3, 37.4)",59.4,"(55.8, 62.9)",18.3,"(16.7, 19.8)",38.0,"(36.8, 39.0)",63.2,"(60.3, 66.2)",19.6,"(17.9, 21.1)",41.2,"(39.9, 42.3)",3.4,"( 3.2, 3.7)",35.6,"(30.4, 40.9)","(39.7419966154, -104.87522395)",NaN
1,1,8001007802,Denver,2016,1152,102.0,67.0,1.35,1.89,"Census Tract 78.02, Adams County, Colorado",1343,1236,597,527,314,314,23,17,62,62,0,0,299,273,48,43,239,194,660,604,782.0,28087.0,1236,54,17,185,169,134,94,113,...,55.3,"(52.4, 58.3)",35.3,"(32.9, 37.8)",8.1,"( 7.4, 8.7)",22.8,"(19.7, 26.0)",18.3,"(16.1, 20.8)",33.5,"(30.3, 36.3)",30.7,"(27.3, 34.8)",12.3,"(11.6, 13.0)",35.2,"(33.9, 36.6)",3.0,"( 2.8, 3.3)",36.9,"(34.1, 39.5)",56.3,"(53.1, 59.6)",19.0,"(17.2, 20.6)",39.4,"(38.2, 40.5)",64.0,"(61.0, 67.2)",21.1,"(19.5, 22.8)",40.9,"(39.6, 42.1)",3.8,"( 3.6, 4.1)",39.0,"(34.1, 43.8)","(39.74199593, -104.856437873)",NaN
2,2,8001007900,Denver,2016,1452,194.0,122.0,2.04,2.72,"Census Tract 79, Adams County, Colorado",1793,1263,810,448,325,279,83,83,71,49,0,0,357,287,147,117,424,176,805,611,871.0,36250.0,1263,9,26,107,197,107,159,106,...,58.6,"(55.9, 61.0)",40.0,"(37.0, 43.2)",7.0,"( 6.4, 7.6)",25.5,"(22.1, 29.2)",20.1,"(17.4, 23.2)",30.4,"(27.6, 33.3)",38.0,"(33.7, 42.4)",10.9,"(10.4, 11.4)",34.1,"(32.9, 35.4)",2.8,"( 2.6, 3.0)",31.8,"(29.1, 34.3)",60.7,"(57.0, 64.7)",16.8,"(15.2, 18.3)",36.7,"(35.5, 37.9)",68.8,"(65.7, 72.0)",18.1,"(16.5, 19.6)",39.4,"(38.0, 40.6)",3.4,"( 3.1, 3.6)",32.7,"(26.6, 38.0)","(39.7489738152, -104.873829752)",NaN
3,3,8001008000,Denver,2016,842,35.0,18.0,0.63,0.69,"Census Tract 80, Adams County, Colorado",1742,743,925,321,377,174,47,26,56,24,0,0,286,166,51,32,621,210,625,281,1032.0,49898.0,743,59,60,87,99,87,52,27,...,66.1,"(63.9, 68.3)",52.4,"(49.6, 55.1)",5.4,"( 4.9, 6.0)",29.6,"(26.1, 33.2)",26.4,"(23.3, 29.6)",22.5,"(20.3, 24.6)",53.6,"(50.1, 57.2)",9.6,"( 9.1, 10.1)",34.7,"(33.7, 35.8)",2.6,"( 2.5, 2.8)",23.9,"(22.0, 25.9)",67.9,"(65.0, 70.6)",12.3,"(11.2, 13.4)",31.4,"(30.5, 32.4)",77.6,"(75.4, 79.7)",13.0,"(12.0, 14.1)",35.8,"(34.7, 37.0)",2.9,"( 2.6, 3.1)",21.0,"(16.5, 26.0)","(39.7493733465, -104.856393321)",NaN
4,4,8001008100,Denver,2016,644,39.0,19.0,0.92,0.96,"Census Tract 81, Adams County, Colorado",673,640,538,512,16,16,0,0,31,28,0,0,30,26,58,58,479,473,89,65,1373.0,35625.0,640,17,32,17,71,42,39,55,...,61.7,"(59.9, 63.5)",60.1,"(57.6, 62.2)",4.4,"( 3.7, 5.8)",36.6,"(29.1, 42.6)",26.8,"(20.7, 31.8)",14.7,"(12.9, 16.9)",64.1,"(60.0, 67.1)",7.4,"( 6.7, 8.5)",35.5,"(34.2, 36.8)",2.6,"( 2.3, 3.0)",17.6

In [5]:
df['ev_rate']=df.Evictions/df.Renters

In [6]:
#remove unnecessary columns 
#add race ratio columns 
df['white_ratio']=df.White/df.totrace
df['black_ratio']=df.Black/df.totrace
df['asian_ratio']=df.Asian/df.totrace
df['latinx_ratio']=df.Latinx/df.totrace
df=df.drop(columns=['totrace','White','Black',
                    'Asian','Latinx'])

In [7]:
#add race rent ratio columns 
df['white_rent_ratio']=df.Rent_WHITE/df.Total_WHITE
df['black_rent_ratio']=df.Rent_BLACK/df.Total_BLACK
df['asian_rent_ratio']=df.Rent_ASIAN/df.Total_ASIAN  
df['ai_ratio']=df.Rent_AI/df.Total_AI

df=df.drop(columns=['Total_WHITE','Total_BLACK','Total_ASIAN',
                    'Total_AI','Rent_WHITE','Rent_BLACK',
                    'Rent_ASIAN','Rent_AI'])

In [8]:
#add household income ratio columns 
df['rb10']=df["rb_10.0"]/df["rb_tot"]
df['rb14.9']=df["rb_14.9"]/df["rb_tot"]
df['rb19.9']=df["rb_19.9"]/df["rb_tot"]
df['rb24.9']=df["rb_24.9"]/df["rb_tot"]
df['rb29.9']=df["rb_29.9"]/df["rb_tot"]
df['rb34.9']=df["rb_34.9"]/df["rb_tot"]
df['rb39.9']=df["rb_39.9"]/df["rb_tot"]
df['rb49.9']=df["rb_49.9"]/df["rb_tot"]
df['rb55']=df["rb_55"]/df["rb_tot"]

df=df.drop(columns=['rb_10.0','rb_14.9','rb_19.9',
                    'rb_24.9','rb_29.9','rb_34.9',
                    'rb_39.9','rb_49.9','rb_55','rb_tot'])

In [9]:
df['unemployed_rate']=df["unemployed"]/df["labor_force"]

In [10]:
df['risk']=np.where(df['Evictions_RR']> 1.5, True, False)

In [15]:
sum(df['risk'])

2118

In [18]:
len(df['risk'])

10851

In [22]:
df2=df.dropna(1)

In [23]:
sum(df2['risk'])

2118

In [24]:
len(df2['risk'])

10851

In [27]:
df2.head(1)

,Unnamed: 0,GEOID,Region,Year,Renters,Tract,Total,Rent,Total_NHPI,Rent_NHPI,Total_OTHER,Rent_OTHER,Total_TWO,Rent_TWO,Total_WHITE_NL,Rent_WHITE_NL,Total_LATINX,Rent_LATINX,rb_nc,totwelf,welf,totpov,povfamh,povnonfamh,totenroll,colenroll,proenroll,HHInc_Total,HHInc_10,HHInc_15,HHInc_20,HHInc_25,HHInc_30,HHInc_35,HHInc_40,HHInc_45,HHInc_50,HHInc_60,HHInc_75,HHInc_100,HHInc_125,HHInc_150,HHInc_200,HHInc_250,HHIncTen_Total,HHIncTenOwn,HHIncTenOwn_5,HHIncTenOwn_10,HHIncTenOwn_15,HHIncTenOwn_20,HHIncTenOwn_25,HHIncTenOwn_35,HHIncTenOwn_50,HHIncTenOwn_75,HHIncTenOwn_100,HHIncTenOwn_150,HHIncTenOwn_151,HHIncTenRent,HHIncTenRent_5,HHIncTenRent_10,HHIncTenRent_15,HHIncTenRent_20,HHIncTenRent_25,HHIncTenRent_35,HHIncTenRent_50,HHIncTenRent_75,HHIncTenRent_100,HHIncTenRent_150,HHIncTenRent_151,labor_force,employed,unemployed,PropValue_50,PropValue_100,PropValue_150,PropValue_200,PropValue_300,PropValue_500,PropValue_Median,risk
0,0,8001007801,Denver,2016,1158,"Census Tract 78.01, Adams County, Colorado",1276,1111,0,0,175,139,28,28,248,172,521,467,17,1276,31,1276,380,165,4114,96,0,1276,171,124,198,141,126,59,75,48,95,61,69,89,9,0,4,7,1276,165,0,25,0,0,9,17,49,19,46,0,0,1111,38,108,124,198,132,168,169,111,43,9,11,2116,1953,163,165,24,12,14,67,38,0,False


In [26]:

def looper(limit):
    cols = ['white_ratio','black_ratio','asian_ratio','latinx_ratio','white_rent_ratio',
          'black_rent_ratio','asian_rent_ratio','ai_ratio','rb10','rb14.9','rb19.9',
         'rb24.9','rb29.9','rb34.9','rb39.9','rb49.9','rb55','unemployed_rate']
    #cols =['black_ratio','black_rent_ratio','rb10','rb14.9','unemployed_rate']
    for i in range(len(cols)):
        data = df2[cols]
        x = sm.add_constant(data1) #constant 
        y = df2['ev_rate'] #target variable 
        model = sm.OLS(y, x) #make the model 
        result = model.fit() #fit your model 
        pvalues = result.pvalues #get all p value frome the result 
        pvalues.drop('const',inplace=True) #remove constant
        pmax = max(pvalues) #get the largest p_value
        if pmax>limit:
            ind = pvalues.idxmax() 
            cols.remove(ind)
        else:
            return result
 
result = looper(0.05)
result.summary()

KeyError: ignored

In [14]:
import xgboost as xgb

In [ ]:
def show_accuracy(a, b, tip):
    acc = a.ravel() == b.ravel()
    print (acc)
    print (tip + 'accuracy_rate：\t', float(acc.sum()) / a.size)

In [ ]:
cols = ['white_ratio','black_ratio','asian_ratio','latinx_ratio','white_rent_ratio',
          'black_rent_ratio','asian_rent_ratio','ai_ratio','rb10','rb14.9','rb19.9',
         'rb24.9','rb29.9','rb34.9','rb39.9','rb49.9','rb55','unemployed_rate']
X = df2[cols]
Y = df2['risk'] 
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=1)  
data_train = xgb.DMatrix(X_train,label=y_train)
data_test = xgb.DMatrix(X_test,label=y_test)
print (data_train)
print (type(data_train))


<class 'xgboost.core.DMatrix'>


In [ ]:
param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softmax','num_class': 3} # logitraw
watchlist = [(data_test, 'eval'), (data_train, 'train')]
n_round = 7
bst = xgb.train(param, data_train, num_boost_round=4, evals=watchlist)  
y_hat = bst.predict(data_test)
show_accuracy(y_hat, y_test, 'XGBoost ')


[0]	eval-merror:0.254237	train-merror:0.125
[1]	eval-merror:0.237288	train-merror:0.079545
[2]	eval-merror:0.220339	train-merror:0.073864
[3]	eval-merror:0.220339	train-merror:0.0625
[ True  True  True False  True  True  True  True False False  True  True
  True  True  True  True  True  True  True  True False  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True False False False False  True False  True  True
  True  True  True False  True  True  True  True False  True  True]
XGBoost accuracy_rate：	 0.7796610169491526


In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=9, max_leaf_nodes=7, n_jobs=1)
rnd_clf.fit(X_train, y_train)
y_hat = rnd_clf.predict(X_test)
print("############Random Forest ###############")
show_accuracy(y_hat, y_test, 'Random Forest')



############Random Forest ###############
[ True  True  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True False  True False False  True  True  True False
  True  True  True False  True  True  True  True False  True  True]
Random Forestaccuracy_rate：	 0.847457627118644


In [ ]:
#bagging
tree = DecisionTreeClassifier(criterion='entropy', max_depth=None)
clf = BaggingClassifier(base_estimator=tree, n_estimators=6, max_samples=1.0, max_features=1.0, bootstrap=True,
                            bootstrap_features=False, n_jobs=1, random_state=1)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
print("##############bagging#############")
show_accuracy(y_hat, y_test, 'bagging ')


##############bagging#############
[ True  True  True False  True  True  True  True False False False  True
  True  True  True  True  True  True  True  True False  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True False  True False False  True False  True False
  True  True  True False False  True  True  True False  True  True]
bagging accuracy_rate：	 0.7457627118644068
